<a href="https://colab.research.google.com/github/ufrpe-ensino/workshop-extracao-informacao/blob/main/notebooks/03_ExtracaoInformacao_Spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spacy: treinando modelos customizados (NER)

## Setup

In [ ]:
!pip install spacy
!spacy download pt_core_news_sm

## Extraindo entidades com o modelo pré-treinado

In [ ]:
import spacy
from spacy import displacy

text = 'Donald Trump, portador do cpf 064.468.404-62, chegou aos EUA no dia 01/06/2019. Donald Trump visitou Barack Obama e George Bush. Além disso, ele esteve em diversos estados do país pilotando uma ferrari.'

my_model = spacy.load('pt_core_news_sm')
document = my_model(text)

print('Original Sentence: %s\n\n' % (text))
    
displacy.render(document, jupyter=True, style='ent')

# Criando seu próprio modelo

## Conjunto de treinamento

In [ ]:
TRAIN_DATA = [('Em 15/07/1988 nasceu essa linda criança', [(3, 13, 'DATA')]),
              ('Rafael Mello, portador do cpf 064.468.404-62, chegou aos EUA', [(0, 12, 'PER'),(30, 44, 'CPF')]),
              ('Data de prisão: 10/01/2018', [(16, 26, 'DATA')]),
              ('No dia 01/02/2016 foi decretada a setença', [(7, 17, 'DATA')]),
              ('A data da festa foi 07/05/2019', [(20, 30, 'DATA')]),
              ('Joao, portador do cpf 123.456.789-65, foi encontrado',  [(0, 4, 'PER'), (22, 36, 'CPF')]),
              ('O cpf 025.412.876-99 pertence a Maria', [(6, 20, 'CPF')])
             ]

In [ ]:
import spacy
from spacy.training import Example
from spacy.tokens import DocBin

nlp = spacy.blank("pt")

# the DocBin will store the example documents
db = DocBin()
examples = []
for text, entities in TRAIN_DATA:
  doc = nlp(text)
  ents = []
  for start, end, label in entities:
    span = doc.char_span(start, end, label=label)
    ents.append(span)
  doc.ents = ents
  db.add(doc)

db.to_disk("./train.spacy")

## Treinando o modelo

In [ ]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

## Usando o modelo treinado

In [ ]:
docnlp_ner = spacy.load("/content/model-best")

In [ ]:
text = 'Donald Trump, portador do cpf 064.468.404-62, chegou aos EUA no dia 01/06/2019. Donald Trump visitou Barack Obama e George Bush. Além disso, ele esteve em diversos estados do país pilotando uma ferrari.'

spacy.displacy.render(docnlp_ner(text), style="ent", jupyter=True) # display in Jupyter

# FLAIR
Utilizaremos inicialmente o Flair Framework para nossa tarefa de NER. É uma biblioteca que implementa o estado da arte em NLP atualmente, além de permitir o uso de modelos pré-treinados de forma bastante prática e simples. Ele utiliza o Pytorch como base para definição da sua arquitetura de redes neurais. 

### Instalando

In [ ]:
!pip install flair

### Testando

In [ ]:
#import commands for flair NER
from flair.data import Sentence
from flair.models import SequenceTagger

### Modelos Pré-treinados (EN)

Você pode ver a lista completa de modelos pré-treinados aqui: https://huggingface.co/models?library=flair&sort=downloads

In [ ]:
#Load NER Model
tagger = SequenceTagger.load('ner')

In [ ]:
#Sample text to run NER
text = 'Jackson is placed in Microsoft located in Redmond'

#passing text to sentence
sentence = Sentence(text)

# Run NER on sentence to identify Entities
tagger.predict(sentence)

# print the entities with below command
for entity in sentence.get_spans('ner'):
    print(entity)

In [ ]:
print(sentence.to_tagged_string())

Testando outra sentença

In [ ]:
#Sample text
text1 = 'Redmond is coming to New York city'

#passing text to sentence
sentence = Sentence(text1)

# Run NER on sentence to identify Entities
tagger.predict(sentence)

# print the entities with below command
for entity in sentence.get_spans('ner'):
    print(entity)

In [ ]:
print(sentence.to_tagged_string())

### Treinando modelo FLAIR em Portugues

O treinamento de modelos do FLAIR é bem custoso, e normalmente obtém melhores resultados utilizando embeddings pré-treinados associados a embeddings específicos para fases forward e backward do algoritmo. 

Até o momento não existem poucos modelos prétreinados disponíveis diretamente no framework, mas algumas iniciativas já estão em andamento (https://github.com/jneto04/ner-pt)

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load tagger
tagger = SequenceTagger.load("noharm-ai/anony")
# make example sentence
sentence = Sentence("FISIOTERAPIA TRAUMATO - MANHÃ  Henrique Dias, 38 anos. Exercícios metabólicos de extremidades inferiores. "\
                    +"Realizo mobilização patelar e leve mobilização de flexão de joelho conforme liberado pelo Dr Marcelo Arocha."\
                    +" Oriento cuidados e posicionamentos.")
# predict NER tags
tagger.predict(sentence)
# print sentence
print(sentence)
# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

In [ ]:
text = 'Donald Trump, portador do cpf 064.468.404-62, chegou aos EUA no dia 01/06/2019. José da Silva visitou Barack Obama e George Bush. Além disso, ele esteve em diversos estados do país pilotando uma ferrari.'
sentence = Sentence(text)
tagger.predict(sentence)

print(sentence)

# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)